# Submit MODIS Global Subset Tool orders using Python

*Author: ORNL DAAC*
*Date: May 29, 2018*
*Contact for the ORNL DAAC: uso@daac.ornl.gov*

### Keywords: MODIS, web service, Python, REST

## Overview

This notebook will demonstrate how to submit a batch of orders to the [MODIS Global Subset Tool](https://modis.ornl.gov/cgi-bin/MODIS/global/subset.pl) for a list of coordinates in a text file using the MODIS Web Services API maintained by the ORNL DAAC. For a full description and usage examples of the web service, please visit the ORNL DAAC's MODIS microsite: https://modis.ornl.gov/data/modis_webservice.html

## Prerequisites:
Python 2 or 3 Libraries: requests, pandas, json, datetime

## Tutorial:

Import libraries and set request URL and headers. Point python to your text file input, formatted in this example with nine columns matching the parameters required by the **subsetOrder** web service function:
```
site_id,product,latitude,longitude,email,start_date,end_date,kmAboveBelow,kmLeftRight
site1,MOD13Q1,35.0,-90.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
site2,MOD13Q1,40.0,-95.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
site3,MOD13Q1,45.0,-100.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
site4,MOD13Q1,50.0,-105.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
site5,MOD13Q1,55.0,-110.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
```
You can of course format your input file however best suits your needs; e.g. the product, email, start_date, end_date, kmAboveBelow, and kmLeftRight are redundant in this example and could be excluded.

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime

url = "https://modis.ornl.gov/rst/api/v1/"
header = {'Accept': 'application/json'} # Use following for a csv response: header = {'Accept': 'text/csv'}

csv = "example_sites.csv"

First, read your file into a *pandas* data frame:

In [2]:
coordinates = pd.read_csv(csv)
coordinates

,site_id,product,latitude,longitude,email,start_date,end_date,kmAboveBelow,kmLeftRight
0,site1,MOD13Q1,35.0,-90.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
1,site2,MOD13Q1,40.0,-95.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
2,site3,MOD13Q1,45.0,-100.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
3,site4,MOD13Q1,50.0,-105.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8
4,site5,MOD13Q1,55.0,-110.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8


Before we can submit orders for the global tool, we must find the MODIS dates nearest to the start and end of our desired time series. We will use the **dates** function to get a list of available MODIS dates for each of our input coordinates:

    /api/v1/{product}/dates

The **dates** function returns a list of available dates for the specified coordinate and MODIS product.

| Parameter          | Description                                                   |
|:-------------------|:--------------------------------------------------------------|
| **product**        | MODIS product code as listed by **products** function         |
| **latitude**       | latitude                                                      |
| **longitude**      | longitude                                                     |

Iterate through the coordinates in the data frame and create lists of MODIS dates. We will add the lists as a new column of the data frame:

In [3]:
# Convert start_date and end_date columns to datetimes
coordinates['start_date'] =  pd.to_datetime(coordinates['start_date'])
coordinates['end_date'] =  pd.to_datetime(coordinates['end_date'])

# Make new columns for MODIS start and end dates 
coordinates['start_MODIS_date'] = '' 
coordinates['end_MODIS_date'] = ''

for index, row in coordinates.iterrows():
    # Submit request
    response = requests.get('https://modis.ornl.gov/rst/api/v1/' + row['product'] + '/dates?latitude=' + str(row['latitude']) + '&longitude='+ str(row['longitude']), headers=header)
    
    # Get dates object as list of python dictionaries
    dates = json.loads(response.text)['dates'] 
    
    # Convert to list of tuples; change calendar_date key values to datetimes
    dates = [(datetime.strptime(date['calendar_date'], "%Y-%m-%d"), date['modis_date']) for date in dates]
    
    # Get MODIS dates nearest to start_date and end_date and add to new pandas columns
    coordinates.loc[index, 'start_MODIS_date'] = min(date[1] for date in dates if date[0] > row['start_date'])
    coordinates.loc[index, 'end_MODIS_date'] = max(date[1] for date in dates if date[0] < row['end_date'])

coordinates

,site_id,product,latitude,longitude,email,start_date,end_date,kmAboveBelow,kmLeftRight,start_MODIS_date,end_MODIS_date
0,site1,MOD13Q1,35.0,-90.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8,A2000049,A2005353
1,site2,MOD13Q1,40.0,-95.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8,A2000049,A2005353
2,site3,MOD13Q1,45.0,-100.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8,A2000049,A2005353
3,site4,MOD13Q1,50.0,-105.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8,A2000049,A2005353
4,site5,MOD13Q1,55.0,-110.0,mcnelisjj@ornl.gov,2000-01-01,2005-12-31,8,8,A2000049,A2005353


Now, we are ready to submit our subset orders. We will use the **subsetOrder** function to pass our subset parameters to the ORNL DAAC's [MODIS Global Subset Tool](https://modis.ornl.gov/cgi-bin/MODIS/global/subset.pl) service:

    /api/v1/{product}/subsetOrder

The **subsetOrder** function returns a unique order identifier (*uid*) that you can use to retreive your order URL. You will also receive an email at the supplied email address once the order has completed processing. Processing times vary based on the size of the order. Most are completed within 30 minutes.

| Parameter          | Description                                                   |
|:-------------------|:--------------------------------------------------------------|
| **product**        | MODIS product code as listed by **products** function         |
| **latitude**       | latitude                                                      |
| **longitude**      | longitude                                                     |
| **email**          | email address for order delivery                              |
| **uid**            | unique order identifier                                       |
| **start_date**     | MODIS start date as listed by **dates** ("AYYYYDOY")          |
| **end_date**       | MODIS end date as listed by **dates** ("AYYYYDOY")            |
| **kmAboveBelow**   | number of kilometers to subset above and below center pixel   |
| **kmLeftRight**    | number of kilometers to subset left and right of center pixel |

Iterate again through the rows of the dataframe and submit orders using the **subsetOrder** function:

In [4]:
# Make list to collect order UIDs
order_uids = []

for index, row in coordinates.iterrows():
    # Build request URL
    requestURL = url + row['product'] + "/subsetOrder?latitude=" + str(row['latitude']) + "&longitude=" + str(row['longitude']) + "&email=" + row['email'] + "&uid=" + row['site_id'] + "&startDate=" + row['start_MODIS_date'] + "&endDate=" + row['end_MODIS_date'] + "&kmAboveBelow=" + str(row['kmAboveBelow']) + "&kmLeftRight=" + str(row['kmLeftRight'])

    # Submit request
    response = requests.get(requestURL, headers=header)
    
    # Append UID to list
    order_uids.append(json.loads(response.text)['order_id'])
    
order_uids

[u'29May2018_14:55:35_037543986L35.0L-90.0S65L65_MOD13Q1_site1',
 u'29May2018_14:55:35_701080305L40.0L-95.0S65L65_MOD13Q1_site2',
 u'29May2018_14:55:36_310255483L45.0L-100.0S65L65_MOD13Q1_site3',
 u'29May2018_14:55:36_941348266L50.0L-105.0S65L65_MOD13Q1_site4',
 u'29May2018_14:55:37_572193963L55.0L-110.0S65L65_MOD13Q1_site5']

If you see a list of strings formatted like those above, your orders were received by the ORNL DAAC! 

As mentioned above, you will receive an email upon completion of your order that will link you to a customized webpage with interactive visualizations and the subset data in CSV and GeoTIFF formats. For more information about the capabilities of the MODIS Global Subset Tool, please visit the ORNL DAAC's MODIS microsite: https://modis.ornl.gov/

You can also link to your orders directly via the order UID:
```
    https://daac.ornl.gov/subsetdata/<order_uid>
    
    e.g. 
    https://daac.ornl.gov/subsetdata/29May2018_14:55:35_037543986L35.0L-90.0S65L65_MOD13Q1_site1
    https://daac.ornl.gov/subsetdata/29May2018_14:55:35_701080305L40.0L-95.0S65L65_MOD13Q1_site2
    https://daac.ornl.gov/subsetdata/29May2018_14:55:36_310255483L45.0L-100.0S65L65_MOD13Q1_site3
    https://daac.ornl.gov/subsetdata/29May2018_14:55:36_941348266L50.0L-105.0S65L65_MOD13Q1_site4
    https://daac.ornl.gov/subsetdata/29May2018_14:55:37_572193963L55.0L-110.0S65L65_MOD13Q1_site5
```

![order_delivery_example](modis_order_delivery_email.png)
Figure 1. Order delivery email for 29May2018_14:45:05_170194826L35.0L-90.0S65L65_MOD13Q1_site1